In [2]:
!pip install autogluon

  Obtaining dependency information for autogluon from https://files.pythonhosted.org/packages/ce/7b/db15f5d33bb0bcdcaead1740ccb7018bc560a9fde14edd177c62f1051844/autogluon-1.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for autogluon.core[all]==1.0.0 from https://files.pythonhosted.org/packages/d6/0b/d2d017b328a2101348c2a07752f57ae6f4c47bbb67e5527b10823f1db55b/autogluon.core-1.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for autogluon.features==1.0.0 from https://files.pythonhosted.org/packages/a5/21/490e0a4310c7aaa22961c5885e2e03f0a6e8097d1d4af0f6c9b22d8fba50/autogluon.features-1.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for autogluon.tabular[all]==1.0.0 from https://files.pythonhosted.org/packages/98/30/8576a70043614aa1137a229916010b8080374fe131aeeee6323a93189b88/autogluon.tabular-1.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for autogluon.multimodal==1.0.0 from https://files.pythonhosted.org/packages/6

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.5.0 requires botocore<1.29.77,>=1.29.76, but you have botocore 1.34.23 which is incompatible.
torchaudio 2.1.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.


## Preprocessing and Modeling

In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np

train_data=pd.read_csv('train.csv')
test_data=pd.read_csv('test.csv')
original_data=pd.read_csv('Churn_Modelling.csv')

train_data=pd.concat([train_data,original_data],axis=0).drop(['id', 'RowNumber'],axis=1).drop_duplicates().dropna()


In [17]:
cols = ['CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender', 'Age', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember','EstimatedSalary', 'Exited']
train_data=train_data.loc[train_data.groupby(cols)['Tenure'].idxmax()].drop_duplicates()

In [19]:
train_data.head(8).T

,87277,140882,160702,1287,1287,4198,4198,8288
CustomerId,15565701,15565701,15565701,15604515,15565701,15632736,15565706,15565706
Surname,Bruno,Chineze,Ferri,Li Fonti,Ferri,Liang,Akobundu,Akobundu
CreditScore,846,705,698,850,698,850,612,663
Geography,Spain,France,France,France,Spain,Germany,Spain,Spain
Gender,Male,Male,Female,Female,Female,Female,Male,Male
Age,49.0,32.0,39.0,32.0,39.0,30.0,35.0,35.0
Tenure,8,6,2,9,9,3,1,4
Balance,0.0,107209.58,161993.89,0.0,161993.89,104911.35,0.0,0.0
NumOfProducts,2,1,1,2,1,2,1,1
HasCrCard,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0


In [24]:
test_data.head().T

,0,1,2,3,4
id,165034,165035,165036,165037,165038
CustomerId,15773898,15782418,15807120,15808905,15607314
Surname,Lucchese,Nott,K?,O'Donnell,Higgins
CreditScore,586,683,656,681,752
Geography,France,France,France,France,Germany
Gender,Female,Female,Female,Male,Male
Age,23.0,46.0,34.0,36.0,38.0
Tenure,2,2,7,8,10
Balance,0.0,0.0,0.0,0.0,121263.62
NumOfProducts,2,1,2,1,1


In [26]:
train_data["Surname_len"]=train_data["Surname"].apply(len)
test_data["Surname_len"]=test_data["Surname"].apply(len)

train=TabularDataset(train_data)
test=TabularDataset(test_data)

In [29]:
automl=TabularPredictor(label="Exited",
                        problem_type="binary",
                        eval_metric="roc_auc",)
automl.fit(train_data=train,presets="best_quality")

No path specified. Models will be saved in: "AutogluonModels\ag-20240120_112648"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels\ag-20240120_112648/ds_sub_fit/sub_fit_ho.
Beginning AutoGluon training ... Time limit = 900s
AutoGluon will save models to "AutogluonModels\ag-20240120_112648/ds_sub_fit/sub_fit_ho"
=================== System Info ===================
AutoGluon Version:  1.

## Prediction and Submission

In [30]:
automl.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.895520,roc_auc,138.625344,2367.835917,0.050027,86.784364,3,True,18
1,CatBoost_BAG_L2,0.894688,roc_auc,96.195613,1774.416626,1.097452,148.790167,2,True,17
2,WeightedEnsemble_L2,0.894599,roc_auc,43.329899,1434.773983,0.047994,54.982387,2,True,12
3,CatBoost_BAG_L1,0.894113,roc_auc,1.360149,1161.098140,1.360149,1161.098140,1,True,7
4,LightGBMXT_BAG_L2,0.893989,roc_auc,102.600598,1662.562829,7.502437,36.936370,2,True,13
5,LightGBMXT_BAG_L1,0.892319,roc_auc,27.533389,56.030204,27.533389,56.030204,1,True,3
6,RandomForestEntr_BAG_L2,0.892319,roc_auc,113.016101,1868.026013,17.917940,242.399554,2,True,16
7,LightGBM_BAG_L2,0.891828,roc_auc,98.369147,1656.041577,3.270986,30.415118,2,True,14
8,RandomForestGini_BAG_L2,0.891284,roc_auc,116.288940,1859.446713,21.190778,233.820255,2,True,15
9,LightGBM_BAG_L1,0.889606,roc_auc,5.362901,22.825492,5.362901,22.825492,1,True,4


In [31]:
prediction=automl.predict_proba(test)

sub_data=pd.read_csv('sample_submission.csv')
sub_data['Exited']=prediction[1]
sub_data[['id','Exited']].to_csv('submission11.csv',index=False)